In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def one_hot_vector(y):
    out = np.zeros((y.shape[0], max(y)+1))
    for i in range(y.shape[0]):
        out[i, y[i]] = 1
    return out

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-08/bt_train.csv")
valid = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-08/bt_valid.csv")

x1_train = train["x1"].values
x2_train = train["x2"].values
y_train = train["label"].values

x1_valid = valid['x1'].values
x2_valid = valid['x2'].values
y_valid = valid['label'].values

# normalize
x1_mean = np.mean(x1_train)
x1_std = np.std(x1_train)
x2_mean = np.mean(x2_train)
x2_std = np.std(x2_train)

x1_train = (x1_train - x1_mean)/ x1_std
x2_train = (x2_train - x2_mean)/ x2_std

x1_valid = (x1_valid - x1_mean)/ x1_std
x2_valid = (x2_valid - x2_mean)/ x2_std



X_train = np.concatenate([x1_train.reshape(-1,1), x2_train.reshape(-1,1)], axis=1)
y_train_onehot = one_hot_vector(y_train)

X_valid = np.concatenate([x1_valid.reshape(-1,1), x2_valid.reshape(-1,1)], axis=1)
X_train

array([[-0.24889612,  0.74760804],
       [-0.74985907,  0.44945807],
       [-0.5145559 , -0.91335007],
       ...,
       [ 0.61938387,  1.6949654 ],
       [ 0.93715397,  0.24593182],
       [ 1.31587098,  0.87455223]])

###  1. Từ code demo hãy cài đặt thêm một module để chọn ra được bộ weights sao cho accuracy trên tập validation là tốt nhất.


In [4]:
def relu(Z):
    
    return np.maximum(0,Z)
def d_relu(Z):
    # tính đạo hàm cho hàm Relu
    return 1*(Z > 0) 

def softmax(Z):
    return np.exp(Z)/ np.sum(np.exp(Z), axis = 0)

def accuracy(y_hat,y_true) :
    return np.sum(y_hat.argmax(axis = 0) == y_true) / len(y_true)

def loss(y_hat, y_true) :
    # tính cross entropy loss cho mỗi vòng epochs
    return np.sum(-y_true * np.log(y_hat.T))/y_true.shape[0]



Khởi tạo $L$ layers trong mạng neuron, ta sẽ cập nhật các layers theo công thức sau.

**Forward :** 
  \begin{align*}
    Z^{[l]} &= W^{[l]}A^{[l-1]} + B^{[l]}   \\
    A^{[l]} &= Relu(Z^{[l]})  \\
    \text{với  } l &\in [0,L - 1]
  \end{align*}

 $Z$ là lớp tuyến tính , $A$ là lớp kích hoạt

 Ở layer thứ 0 là giá trị đầu vào nên $A^{[0]} = X$

 Ở layer cuối cùng , ta sẽ cập nhật $A^{[L]}$ bằng hàm softmax : $A^{[L]} = softmax(Z^{[L]})$

Trong đó $W^{[l]}$ và $B^{[l]}$ là các tham số weight và biases ứng với layer $l$ 

**Backward** :

  \begin{align*}
    dZ^{[l]}&= dA^{[l]} * relu'(Z^{[l]}) \\
    dW^{[l]} &= \dfrac{1}{m} dZ^{[l]} A^{[l-1]T}  \\
    dB^{[l]} &= \dfrac{1}{m}\sum\limits_{i=1}^{m}dZ^{[l](i)} \\
    dA^{[l-1]} &= W^{[l]T}dZ^{[l]} \\
    \text{với  } l &\in [0,L - 1]
  \end{align*}

Trong đó $m$ là số sample , $(i)$ là sample thứ i

Ở layer cuối cùng , $dZ^{[L]} = A^{[L]} - Y $ \\
Ở layer đầu tiên  , $dZ^{[1]} = dA^{[1]} * relu'(X)$

Sau khi **forward** và **backward** ta sẽ cập nhật cho $W$ và $B$ bằng **Gradient Descent** :

\begin{cases}
  W^{[l]}  &:=   W^{[l]} - \alpha *  dW^{[l]} \\
  B^{[l]}  &:=   B^{[l]} - \alpha *  dB^{[l]}  \\
\end{cases}





In [5]:
class Neural_Net :
  
  def __init__ (self, n_layers , list_N_L):

    '''
    Tham số :
    n_layers -- Số hidden layers khởi tạo
    list_N_L -- List các số neuron tương ứng với layers khởi tạo  
    '''
    assert n_layers == len(list_N_L)
    self.n_layers = n_layers
    self.list_N_L = list_N_L
  
  def fit (self, X_train , y_train , n_epochs , lr ) :
    '''
    Tìm ra bộ Weight và bias tốt nhất trên tập train 
    Tham số :
    X_train -- Dữ liệu cần train
    y_train -- Label của tập train
    n_epochs -- Số epoch để chạy  
    lr   -- tốc độ học
    '''
    self.X_train = X_train
    self.y_train = y_train 
    caches_update_para = []
    self.y_train_onehot = one_hot_vector(y_train)
    cache_init_parameter  = self._init_para()

    A_L , forward_caches = self._forward(cache_init_parameter , self.X_train)
    backward_caches = self._backward(A_L,forward_caches)

    print('Epoch 1 :    ---------- Loss : {:.7f}------------ Accuracy : {:.7f}'.format(loss(A_L,self.y_train_onehot) , accuracy(A_L, self.y_train) ))
    update_para = self._update_par(forward_caches, backward_caches , lr)

    caches_update_para.append(update_para)
    for i in range(2, n_epochs + 1) :
        A_L1 , forward_caches1 = self._forward(update_para , self.X_train)
        if i % 1000 == 0 :
          print('Epoch {} : ---------- Loss : {:.7f}------------ Accuracy : {:.7f}'.format(i, loss(A_L1,self.y_train_onehot) , accuracy(A_L1, y_train)))
        backward_caches1 =  self._backward(A_L1,forward_caches1)
        update_para = self._update_par(forward_caches1, backward_caches1 , lr)
        caches_update_para.append(update_para)
    self.update_para = caches_update_para[-2]
    
  def summary(self ,X_train ,  y_train) :
    '''
    Mô ta các kích thước và số tham số của mô hình 
    Tham số :
    X_train -- Dữ liệu cần train
    y_train -- Label của tập train
    '''
    n_samples = X_train.shape[0]
    input_feature = [X_train.shape[1]]
    output_class = [len(np.unique(y_train))]
    list_para = input_feature +  self.list_N_L + output_class
    count_para = 0
    print('Cấu trúc model : ')
    print()
    print('Số sample : {}'.format(n_samples))
    print('Số feature : {}'.format(input_feature[0]))
    print('_'*65)
    print('Layer (type) \t \t \t Output shape \t \tParam # '  )
    print('='*65)
    for i in range(len(list_para) - 2 ) :
      print('_'*65)
      current_para = list_para[i]*list_para[i+1]+list_para[i+1] 
      print('dense_{} (Dense) \t \t (None,{}) \t \t {}'.format(i+1,self.list_N_L[i],current_para))
      count_para += current_para
    print('_'*65)
    current_para = list_para[-2]*list_para[-2+1]+list_para[-2+1]
    count_para += current_para
    print('dense_{} (Dense) \t \t (None,{}) \t \t {}'.format(self.n_layers +1,output_class[0],current_para))
    print('='*65)
    print('Total params : {}'.format(count_para))
    print('Trainable params : {}'.format(count_para))
    print('Non-trainable params: 0')
    print('_'*65)

  def predict(self, X_test) :
    '''
    Tham số :
    X_test -- Dữ liệu test
    Return :
    Trả về label dự đoán của tập test 
    '''
    self.A_L = self._forward(self.update_para, X_test)
    return self.A_L.argmax(axis = 0)

  def accuracy(self,X_test , y_test) :
    '''
    Tham số :
    X_test -- Dữ liệu test
    y_test -- Label của tập test
    Return : 
    Trả về accuracy giữa label dự đoán và label thực tế
    '''
    A_L , forward_caches  = self._forward(self.update_para  , X_test)
    return np.sum(A_L.argmax(axis = 0)  == y_test) / len(y_test)

  def _init_para (self):
    '''
    Khởi tạo các tham số cho Epoch đầu tiên

    Return :
    Hàm trả về một list bao gồm các tuple chứa (W,b) cho từng lớp layers
    '''
    np.random.seed(3)
    N_0 = self.X_train.shape[1]
    cache_init_parameter = [] # biến lưu các tham số
    # tạo tham số cho layer đầu tiên
    W = 0.1 * np.random.rand(self.list_N_L[0], N_0)
    b = np.zeros((self.list_N_L[0],1))
    cache_init_parameter.append((W,b))
    
    # tạo tham số cho các hidden layers
    for i in range(1, self.n_layers) :
        W = 0.1 * np.random.rand(self.list_N_L[i] , self.list_N_L[i-1])
        b = np.zeros((self.list_N_L[i],1))
        cache_init_parameter.append((W,b))
        
    # tạo tham số cho layer cuối cùng    
    n_classes = len(np.unique(self.y_train))    
    W = 0.1 * np.random.rand( n_classes , self.list_N_L[-1])
    b = np.zeros((n_classes,1) )
    cache_init_parameter.append((W,b))
    return cache_init_parameter

  def _forward(self, caches_parameter , X_train) :
    '''
    Tính lan truyền xuôi từ input đầu vào 
    Tham số :
    caches_parameter : list chứa các (W,b) cho từng layers
    X_train : Dữ liệu train

    Return :
    Trả về A_L ở layer cuối cùng và một list gồm các tham số (W,b,Z) tương ứng với mỗi layer
    '''
    forward_caches = [] # list lưu trữ các biến (W,b,Z)
    # forward đối với layer Input
    Z_1 = caches_parameter[0][0] @ X_train.T + caches_parameter[0][1]
    A_prev = relu(Z_1)
    forward_caches.append((caches_parameter[0][0] , caches_parameter[0][1] , Z_1 ))
    # forward giữa các hidden layers
    L = len(caches_parameter)
    for l in range(1, L - 1) :
        Z_l = caches_parameter[l][0] @ A_prev + caches_parameter[l][1]
        forward_caches.append((caches_parameter[l][0] ,  caches_parameter[l][1] , Z_l))
        A_prev = relu(Z_l)
    # forward ở layer cuối cùng     
    Z_L = caches_parameter[-1][0] @ A_prev + caches_parameter[-1][1]
    forward_caches.append((caches_parameter[-1][0] ,  caches_parameter[-1][1] , Z_L))
    A_prev = softmax(Z_L)
    
    return  A_prev , forward_caches


  def _backward(self, A_L , caches) :
    '''
    Tính lan truyền ngược  
    Tham số :
    A_L : A_L ở layer cuối cùng ở bước forward
    caches : list gồm các tham số (W,b,Z) ở các layer tương ứng tính được ở bước forward 
    Return :
    Trả về list gồm các (dW,db) ở các layer tương ứng được dùng để cập nhật cho các tham số (W,b)
    '''
    m = len(self.y_train)
    d_caches = []
    y_train_onehot = one_hot_vector(self.y_train)

    # backward layer cuối cùng
    d_Z_L  = A_L - y_train_onehot.T
    d_W_L  = 1/m * d_Z_L @ relu(caches[-2][2]).T
    d_b_L  = 1/m * np.sum(d_Z_L , axis = 1 , keepdims = True)
    d_A_prev = caches[-1][0].T @ d_Z_L 
    d_caches.append((d_W_L,d_b_L))
    # backward giữa các hidden layers
    L = len(caches)
    for i in range(L-2 , 0 , -1) :
        d_Z = d_A_prev * d_relu(caches[i][2])
        d_W = 1/m * d_Z @ relu(caches[i-1][2].T)
        d_b = 1/m * np.sum(d_Z , axis = 1 , keepdims = True)
        d_caches.append((d_W,d_b))
        d_A_prev = caches[i][0].T @ d_Z 
    # backward ở layer Input
    d_Z = d_A_prev * d_relu(caches[0][2])
    d_W = 1/m * d_Z @ X_train
    d_b = 1/m * np.sum(d_Z , axis = 1 , keepdims = True)
    d_caches.append((d_W,d_b))

    return d_caches[::-1]    

  def _update_par (self, caches, d_caches , lr) :
    '''
    Cập nhật các parameter sau mỗi epoch 
    Tham số :
    caches : list chứa các (W,b,Z) cho từng layers ở bước forward
    d_caches :  list chứa các (dW,db) cho từng layers ở bước backward
    Return :
    Trả về list các (W,b) vừa được cập nhật 
    '''
    assert len(caches) == len(d_caches)
    new_para = []
    for i in range(len(caches)) :
        new_W = caches[i][0] - lr* d_caches[i][0]
        new_b = caches[i][1] - lr* d_caches[i][1]
        new_para.append((new_W , new_b))
    return new_para



In [6]:
model = Neural_Net(3,[3,4,5])
# ở đây ta sẽ chọn 3 hidden layers mỗi layers có số neuron lần lượt là 3 ,4, 5 và layers là softmax để phân loại 3 class
model.summary(X_train,y_train)

Cấu trúc model : 

Số sample : 900
Số feature : 2
_________________________________________________________________
Layer (type) 	 	 	 Output shape 	 	Param # 
_________________________________________________________________
dense_1 (Dense) 	 	 (None,3) 	 	 9
_________________________________________________________________
dense_2 (Dense) 	 	 (None,4) 	 	 16
_________________________________________________________________
dense_3 (Dense) 	 	 (None,5) 	 	 25
_________________________________________________________________
dense_4 (Dense) 	 	 (None,3) 	 	 18
Total params : 68
Trainable params : 68
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(X_train,y_train , 20000, 0.2)

Epoch 1 :    ---------- Loss : 1.0986584------------ Accuracy : 0.2455556
Epoch 1000 : ---------- Loss : 0.4174864------------ Accuracy : 0.8733333
Epoch 2000 : ---------- Loss : 0.3545076------------ Accuracy : 0.8844444
Epoch 3000 : ---------- Loss : 0.2704845------------ Accuracy : 0.9000000
Epoch 4000 : ---------- Loss : 0.2693621------------ Accuracy : 0.8988889
Epoch 5000 : ---------- Loss : 0.2690592------------ Accuracy : 0.8988889
Epoch 6000 : ---------- Loss : 0.2580326------------ Accuracy : 0.9033333
Epoch 7000 : ---------- Loss : 0.2576201------------ Accuracy : 0.9022222
Epoch 8000 : ---------- Loss : 0.2574660------------ Accuracy : 0.9022222
Epoch 9000 : ---------- Loss : 0.2574036------------ Accuracy : 0.9033333
Epoch 10000 : ---------- Loss : 0.2573486------------ Accuracy : 0.9033333
Epoch 11000 : ---------- Loss : 0.2573101------------ Accuracy : 0.9033333
Epoch 12000 : ---------- Loss : 0.2572700------------ Accuracy : 0.9033333
Epoch 13000 : ---------- Loss : 0.2

In [8]:
print('Accuracy trên tập valid : ',model.accuracy(X_valid, y_valid))

Accuracy trên tập valid :  0.64


### 2. Từ bộ dữ liệu bên dưới hãy cài  đặt backpropagation cho bài toán phân biệt ung thư vú. Hãy tự chọn số layers và số nodes mà mình cho là thích hợp, cũng như là nêu ra số layers và số nodes của mỗi layer mà mình đã chọn. Tính accuracy trên tập training.

In [9]:
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data  
y = breast_cancer.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.2, random_state=0)

X_mean=np.mean(X_train)
X_std=np.std(X_train)

X_valid=(X_valid-X_mean)/X_std
X_train=(X_train-X_mean)/X_std

In [10]:
model = Neural_Net(4,[2,2,2,2])
# ở đây ta sẽ chọn 4 hidden layers mỗi layers có 2 neurons và layer cuối cùng là layer softmax để phân loại 2 class
model.summary(X_train,y_train)


Cấu trúc model : 

Số sample : 455
Số feature : 30
_________________________________________________________________
Layer (type) 	 	 	 Output shape 	 	Param # 
_________________________________________________________________
dense_1 (Dense) 	 	 (None,2) 	 	 62
_________________________________________________________________
dense_2 (Dense) 	 	 (None,2) 	 	 6
_________________________________________________________________
dense_3 (Dense) 	 	 (None,2) 	 	 6
_________________________________________________________________
dense_4 (Dense) 	 	 (None,2) 	 	 6
_________________________________________________________________
dense_5 (Dense) 	 	 (None,2) 	 	 6
Total params : 86
Trainable params : 86
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train, y_train , 20000, 0.1)

Epoch 1 :    ---------- Loss : 0.6931463------------ Accuracy : 0.3626374
Epoch 1000 : ---------- Loss : 0.6549190------------ Accuracy : 0.6373626
Epoch 2000 : ---------- Loss : 0.6549182------------ Accuracy : 0.6373626
Epoch 3000 : ---------- Loss : 0.6549167------------ Accuracy : 0.6373626
Epoch 4000 : ---------- Loss : 0.6549133------------ Accuracy : 0.6373626
Epoch 5000 : ---------- Loss : 0.6549037------------ Accuracy : 0.6373626
Epoch 6000 : ---------- Loss : 0.6548495------------ Accuracy : 0.6373626
Epoch 7000 : ---------- Loss : 0.2273435------------ Accuracy : 0.9098901
Epoch 8000 : ---------- Loss : 0.1668665------------ Accuracy : 0.9230769
Epoch 9000 : ---------- Loss : 0.1635580------------ Accuracy : 0.9340659
Epoch 10000 : ---------- Loss : 0.1592793------------ Accuracy : 0.9340659
Epoch 11000 : ---------- Loss : 0.1557558------------ Accuracy : 0.9318681
Epoch 12000 : ---------- Loss : 0.1525407------------ Accuracy : 0.9340659
Epoch 13000 : ---------- Loss : 0.1

In [12]:
print('Accuracy trên tập train : ',model.accuracy(X_train, y_train))
print('Accuracy trên tập valid : ',model.accuracy(X_valid, y_valid))

Accuracy trên tập train :  0.945054945054945
Accuracy trên tập valid :  0.9122807017543859
